In [1]:
# © 2023 Nicolas Bosc <nbosc@ebi.ac.uk>

# Extract PK parameters from ChEMBL assay descriptions
<b>Prerequisite:</b> an accessible Oracle instance of ChEMBL

<b>Objective:</b> Using regex to extract PK/PD parameters (e.g. dose, time, route of admistration etc.) from ChEMBL assay descriptions.

<b>Description:</b>
- The first part queries the ChEMBL database for the relevant data and extract some PK/PD parameters from the assay descriptions
- The second part applies some sanity checks, and standardisation rules to extracted parameters prior to loading the data in ChEMBL.

<b>NB:</b> This notebook does not describe the loading part.

In [10]:
import pandas as pd, numpy as np
import re
import pk_parameters_regex as pk_regex

from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

#### notebook settings
# pd.options.display.max_rows= 100
# pd.options.display.max_columns = 50
# pd.options.display.max_colwidth = 100

## Direct ChEMBL database access 

In [11]:
engine = create_engine(f"oracle+oracledb://{username}:{password}@{hostname}:{port}")

## Functions 

# PART 1

## Regex 

In [12]:
dose_unit = pk_regex.dose_unit
time_of_measurement = pk_regex.time_of_measurement
route_of_administration = pk_regex.route_of_administration
tissue = pk_regex.tissue

##  Legacy data extracted from assay description

#### The PK parameters (time, route of administration, dose etc.) might be written in the assay description and they can be extracted using regular expressions.

#### Legacy data definition
- Data from CHEMBL 33
- standard_type = Cmax or AUC


In [13]:
sql='''
SELECT a.activity_id, a.standard_type,
       b.assay_id,b.description, b.assay_type
FROM
    CHEMBL_33.activities a
    JOIN CHEMBL_33.assays b ON a.assay_id = b.assay_id
WHERE
    lower(a.standard_type) IN ('cmax', 'auc')
'''
df_pk = pd.read_sql(sql, engine)
df_pk.shape

(61779, 5)

In [14]:
df_cmax = df_pk[df_pk.standard_type=='Cmax']
df_cmax.shape

(24692, 5)

In [15]:
df_auc= df_pk[df_pk.standard_type=='AUC']
df_auc.shape

(37087, 5)

In [16]:
df_cmax.activity_id.nunique()

24692

In [17]:
df_auc.activity_id.nunique()

37087

#### Use the regex to get the PK parameters

In [22]:
df_cmax = df_cmax.assign(
    dose_unit=df_cmax.apply(lambda x: pk_regex.find_dose_unit(dose_unit, x['description']), axis=1),
    route=df_cmax.apply(lambda x: pk_regex.find_route(route_of_administration, x['description']), axis=1),
    tissue=df_cmax.apply(lambda x: pk_regex.find_tissue(tissue, x['description']),axis=1),
    )

In [23]:
df_auc = df_auc.assign(
    dose_unit=df_auc.apply(lambda x: pk_regex.find_dose_unit(dose_unit, x['description']), axis=1),
    route=df_auc.apply(lambda x: pk_regex.find_route(route_of_administration, x['description']), axis=1),
    time=df_auc.apply(lambda x: pk_regex.find_time(time_of_measurement, x['description']), axis=1),
    tissue=df_auc.apply(lambda x: pk_regex.find_tissue(tissue, x['description']),axis=1),
    )

# Part 2

In [25]:
df_auc = pk_regex.remove_low_quality_pk_data(df_auc)

DATA FILTERING - START

Before curation, there are 37087 AUC data points.
1150 AUC data points have an ambiguous route of administration and were excluded.
2803 AUC data points have no dose information but were kept.
228 AUC data points have a dose range and they have been excluded.
14908 AUC data points have no time and they have been excluded
After curation, there are now 20800 AUC data points

DATA FILTERING - END


In [26]:
df_cmax = pk_regex.remove_low_quality_pk_data(df_cmax)

DATA FILTERING - START

Before curation, there are 24692 Cmax data points.
1143 Cmax data points have an ambiguous route of administration and were excluded.
1587 Cmax data points have no dose information but were kept.
119 Cmax data points have a dose range and they have been excluded.
After curation, there are now 23402 Cmax data points

DATA FILTERING - END


---

#### Merge Cmax and AUC data 

In [27]:
df_pk = pd.concat([df_cmax, df_auc])

In [28]:
df_pk.shape

(44202, 9)

In [29]:
df_pk

,activity_id,standard_type,assay_id,description,assay_type,dose_unit,route,tissue,time
2,396193,Cmax,5052,Cmax value in rat after oral administration at...,A,10 mg/kg,oral,,NaN
3,448177,Cmax,8533,Cmax after 10 mg/kg oral administration in rat,A,10 mg/kg,oral,,NaN
4,457279,Cmax,8533,Cmax after 10 mg/kg oral administration in rat,A,10 mg/kg,oral,,NaN
5,442836,Cmax,8533,Cmax after 10 mg/kg oral administration in rat,A,10 mg/kg,oral,,NaN
6,441388,Cmax,8533,Cmax after 10 mg/kg oral administration in rat,A,10 mg/kg,oral,,NaN
...,...,...,...,...,...,...,...,...,...
61764,24668593,AUC,2187028,AUC (0 to infinity) in Sprague-Dawley rat at 1...,A,1 mg/kg,iv,,0 to infinity
61765,24668595,AUC,2187028,AUC (0 to infinity) in Sprague-Dawley rat at 1...,A,1 mg/kg,iv,,0 to infinity
61773,24849830,AUC,2233905,AUC (0 to 24 hrs) in Sprague-Dawley rat at 1 m...,A,1 mg/kg,iv,,AUC (0 to 24 hrs)
61775,24790937,AUC,2219010,"AUClast in Sprague-Dawley rat at 2.5 mg/kg, iv...",A,2.5 mg/kg,iv,,up to 24 hrs


#### Replace missing values by None

In [30]:
df_pk = df_pk.replace({'': None}).replace({np.nan: None})

#### Normalise the route of administration

We use the route of administration vocabulary than already used in ChEMBL

In [31]:
route_mapping = {'po':'ORAL','Po':'ORAL','PO':'ORAL','p.o.':'ORAL','Oral':'ORAL','orally':'ORAL'
                 ,'peroral':'ORAL','Peroral':'ORAL','perorally':'ORAL'
                 ,'paroral':'ORAL'
                 ,'Preoral':'ORAL'
                 ,'iv':'INTRAVENOUS','intravenously':'INTRAVENOUS','i.v.':'INTRAVENOUS','IV':'INTRAVENOUS'
                 ,'ip':'INTRAPERITONEAL','i.p.':'INTRAPERITONEAL','intraperitoneally':'INTRAPERITONEAL'
                 ,'sc':'SUBCUTANEOUS'
                 ,'ig':'INTRAGASTRIC','IG':'INTRAGASTRIC','intragastrically':'INTRAGASTRIC'
                 ,'inhalation':'RESPIRATORY (INHALATION)'
                 ,'im':'INTRAMUSCULAR','IM':'INTRAMUSCULAR','intramuscularly':'INTRAMUSCULAR'
                 ,'intraduodenally':'INTRADUODENAL'
                 ,'intranasal':'NASAL', 'intranasally':'NASAL'
                 ,'intrajejunal':'INTRAILEAL'
                 ,'eye':'OPHTHALMIC','eyes':'OPHTHALMIC','intraocular':'INTRAOCULAR','intraocularly':'INTRAOCULAR'
                 ,'i.c.v':'INTRATHECAL'}

In [32]:
df_pk=df_pk.replace({"route": route_mapping})
df_pk['route'] = df_pk['route'].str.upper()

In [33]:
df_pk.route.unique()

array(['ORAL', 'INTRAVENOUS', None, 'INTRAPERITONEAL', 'SUBCUTANEOUS',
       'INTRADUODENAL', 'INTRAMUSCULAR', 'INTRAGASTRIC', 'OPHTHALMIC',
       'NASAL', 'SUBLINGUAL', 'NASOGASTRIC', 'RESPIRATORY (INHALATION)',
       'INTRAILEAL', 'INTRAOCULAR', 'INTRATHECAL'], dtype=object)

#### Normalise tissue 

In [34]:
df_pk = df_pk.assign(tissue=df_pk.tissue.str.lower().str.strip())

tissue_of_measurement = {
                        'csf':'cerebrospinal fluid'    
}

df_pk=df_pk.replace({"tissue": tissue_of_measurement})

df_pk.tissue.unique()

array([None, 'plasma', 'blood', 'spleen', 'lung', 'brain',
       'cerebrospinal fluid', 'portal vein', 'serum', 'heart', 'prostate',
       'biliary fluid', 'liver', 'retina', 'choroid', 'cornea', 'thigh',
       'cage fluid', 'muscle', 'kidney', 'skin', 'interstitial fluid',
       'adipose tissue', 'hypothalamus', 'epithelial lining fluid',
       'peritoneal fluid', 'alveolar cell',
       'lung epithelial lining fluid', 'thigh | interstitial fluid',
       'tumor', 'urine', 'tumor | liver', 'kidney | tumor',
       'heart | tumor', 'body fluid', 'blood | portal vein', 'jejunum',
       'muscle | thigh', 'large intestine', 'small intestine', 'fat',
       'mesenteric lymph', 'extracellular fluid', 'sciatic nerve',
       'stomach', 'blood | liver', 'intestine', 'serum | tumor'],
      dtype=object)

#### Format data 

In [36]:
df_pk = df_pk.melt(id_vars=['activity_id','standard_type','assay_id','description','assay_type'],
             value_vars=['route','time','dose_unit','tissue'])

In [37]:
df_pk.variable.unique()

array(['route', 'time', 'dose_unit', 'tissue'], dtype=object)

In [38]:
df_pk = df_pk.replace('dose_unit','DOSE')
df_pk = df_pk.replace('tissue','TISSUE')
df_pk = df_pk.replace('route','ROUTE')
df_pk = df_pk.replace('time','TIME')

In [39]:
df_pk.variable.unique()

array(['ROUTE', 'TIME', 'DOSE', 'TISSUE'], dtype=object)

In [40]:
df_pk = df_pk.rename({'value':'text_value', 'variable':'type'},axis=1)

In [41]:
df_pk = df_pk.assign(dose_unit=df_pk.text_value)

df_pk.loc[df_pk['type']!='DOSE','dose_unit'] = None
df_pk.loc[df_pk['type']=='DOSE','text_value'] = None

In [44]:
df_pk['value'], df_pk['units'] = zip(*df_pk.dose_unit.apply(lambda x: pk_regex.split_dose_unit(x)))
df_pk = df_pk.drop('dose_unit', axis=1)

In [45]:
df_pk.shape

(176808, 9)

In [46]:
df_pk = df_pk[~((df_pk.text_value.isnull())&(df_pk.value.isnull())&(df_pk.units.isnull()))].sort_values('activity_id')

In [47]:
df_pk.shape

(112452, 9)

#### AUC / Cmax stats 

In [48]:
df_pk[df_pk.standard_type=='Cmax']['activity_id'].nunique()

22792

In [49]:
df_pk[df_pk.standard_type=='Cmax'].groupby('type')['activity_id'].count().reset_index()

,type,activity_id
0,DOSE,21815
1,ROUTE,21260
2,TISSUE,5221


In [50]:
df_pk[df_pk.standard_type=='AUC']['activity_id'].nunique()

20800

In [51]:
df_pk[df_pk.standard_type=='AUC'].groupby('type')['activity_id'].count().reset_index()

,type,activity_id
0,DOSE,20029
1,ROUTE,19754
2,TIME,20800
3,TISSUE,3573


#### Normalise AUC time 

In [52]:
df_time = pd.read_csv('chembl32_regex_time_standardised_2023-05-05.csv')

In [53]:
df_time.relation.value_counts()

relation
<=    251
0      62
Name: count, dtype: int64

In [54]:
df_time.loc[~df_time.text_value.isnull(),'relation'] = None

In [55]:
df_time.relation.value_counts()

relation
<=    236
Name: count, dtype: int64

In [56]:
df_pk[(df_pk['type']=='TIME')].shape

(20800, 9)

In [57]:
df_pk = pd.concat(
            [df_pk[(df_pk['type']!='TIME')]
                , df_pk[(df_pk['type']=='TIME')].drop(
                       ['value','units']
                       , axis=1
                        ).merge(
                               df_time[['time','relation','value','units','text_value']]
                                , left_on = 'text_value'
                                , right_on='time'
                                , how='left').rename(
                                    {'text_value_y':'text_value'},axis=1).drop(
                                        ['text_value_x','time'],axis=1)
            ]
        )
df_pk = df_pk[~((df_pk.text_value.isnull())&(df_pk.value.isnull()))]
df_pk.shape

(112393, 10)

In [58]:
df_pk.loc[(df_pk['type']=='DOSE'),'relation'] = '='

In [59]:
df_pk[df_pk.standard_type=='Cmax']['activity_id'].nunique()

22792

In [60]:
df_pk.relation.value_counts()

relation
=     41844
<=     9951
Name: count, dtype: int64

In [61]:
def format_time_range(two_row_df):
    lower_time = two_row_df[two_row_df.value == two_row_df.value.min()]
    upper_time = two_row_df[two_row_df.value == two_row_df.value.max()]
    lower_time = lower_time.replace({'TIME':'TIME_LOWER'})
    upper_time = upper_time.replace({'TIME':'TIME_UPPER'})
    return(pd.concat([lower_time, upper_time]))
    

#### Rearrange time ranges so text_value does not contains any numerical vales (with the exception of 0-last) 

In [62]:
df_time_range = df_pk[(df_pk['type'] == 'TIME') & (~df_pk.text_value.isnull()) & (~df_pk.units.isnull())]

df_time_range['text_value'] = df_time_range.text_value.apply(lambda x: x.split(' - '))
df_time_range = df_time_range.explode('text_value')
df_time_range.loc[:, 'value'] = df_time_range.text_value
df_time_range.loc[:, 'text_value'] = None
df_time_range = df_time_range.groupby('activity_id').apply(lambda x : format_time_range(x)).droplevel(0)

df_pk = pd.concat([
                    df_pk[~((df_pk['type'] == 'TIME') & (~df_pk.text_value.isnull()) & (~df_pk.units.isnull()))]
                    , df_time_range
                    ])
df_pk.shape

(112775, 10)

In [63]:
df_pk = df_pk.replace({np.nan: None})

In [64]:
df_pk.text_value.unique()

array(['ORAL', None, 'blood', 'INTRAVENOUS', 'plasma', 'SUBCUTANEOUS',
       'cerebrospinal fluid', 'brain', 'INTRAPERITONEAL', 'INTRADUODENAL',
       'portal vein', 'tumor', 'INTRAMUSCULAR', 'heart', 'liver',
       'kidney', 'spleen', 'serum', 'lung', 'INTRAGASTRIC', 'prostate',
       'hypothalamus', 'biliary fluid', 'OPHTHALMIC', 'retina', 'choroid',
       'cornea', 'thigh', 'skin', 'NASAL', 'INTRAOCULAR',
       'epithelial lining fluid', 'cage fluid', 'muscle',
       'adipose tissue', 'interstitial fluid', 'SUBLINGUAL',
       'blood | liver', 'peritoneal fluid', 'alveolar cell',
       'lung epithelial lining fluid', 'body fluid', 'NASOGASTRIC',
       'thigh | interstitial fluid', 'urine', 'intestine',
       'mesenteric lymph', 'blood | portal vein', 'extracellular fluid',
       'heart | tumor', 'sciatic nerve', 'INTRAILEAL', 'small intestine',
       'stomach', 'RESPIRATORY (INHALATION)', 'INTRATHECAL', 'jejunum',
       'muscle | thigh', 'large intestine', 'fat', 'tumor

In [65]:
df_pk.type.unique()

array(['ROUTE', 'DOSE', 'TISSUE', 'TIME', 'TIME_LOWER', 'TIME_UPPER'],
      dtype=object)

### Export flat file

In [114]:
df_pk.to_excel(f'chembl33_cmax_auc_legacy.xlsx', index=False)